# ETL test: load 10 painters form SPARQL endpoint, extract some attributes (name, best artwork) and save to .csv file

Technically counts as an ETL

In [ ]:
import requests
import json
import pandas as pd

In [ ]:
WIKIDATA_SPARQL_ENDPOINT = 'https://query.wikidata.org/sparql'


In [ ]:
def fetch_french_painters_data(limit=10):
    query = f'''
    SELECT DISTINCT ?painter ?painterLabel ?artwork ?artworkLabel
    WHERE {{
        ?painter wdt:P106 wd:Q1028181;
                 wdt:P27 wd:Q142.
        ?artwork wdt:P170 ?painter.
        SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
    }}
    LIMIT {limit}
    '''

    headers = {
        'Accept': 'application/sparql-results+json'
    }

    response = requests.get(WIKIDATA_SPARQL_ENDPOINT, headers=headers, params={'query': query})

    if response.status_code == 200:
        return json.loads(response.text)
    else:
        print(f'Error fetching data: {response.status_code}')
        return None


In [ ]:
def transform_french_painters_data(data):
    results = data['results']['bindings']
    transformed_data = []

    for result in results:
        transformed_data.append({
            'painter': result['painterLabel']['value'],
            'artwork': result['artworkLabel']['value'],
            'painter_url': result['painter']['value'],
            'artwork_url': result['artwork']['value'],
        })

    return transformed_data


In [ ]:
def save_data_to_csv(data, file_name='french_painters_data.csv'):
    df = pd.DataFrame(data)
    df.to_csv(file_name, index=False)
    print(f'Successfully saved data in {file_name}')


In [ ]:
def main():
    raw_data = fetch_french_painters_data()
    if raw_data:
        transformed_data = transform_french_painters_data(raw_data)
        save_data_to_csv(transformed_data)

if __name__ == '__main__':
    main()


Successfully saved data in french_painters_data.csv
